In [3]:
import pandas as pd
import numpy as np
import glob

import xml.etree.ElementTree as ET

In [4]:
def dataset_xml_reader_sep(routes, dataset):

    questions=[]
    ref_answers=[]
    stud_answers=[]

    for i_dtst,path in enumerate(routes):
        files = glob.glob(path)
        for name in files:
            tree = ET.parse(name)
            root = tree.getroot()
            dtset=dataset[i_dtst]
            quest_id=root.attrib["id"]
            question=root[0].text
            try:
                qtype=root.attrib["qtype"]
            except:
                qtype=None
            try:
                stype=root.attrib["stype"]
            except:
                stype=None

            grade_st=[branch.attrib["accuracy"] for branch in root[2]]
            id_st=[branch.attrib["id"] for branch in root[2]]
            answers_st=[branch.text for branch in root[2]]
            answers_ref=[branch.text for branch in root[1]]# If there are more of 1 reference answer it will give an array of an answers. Make sure you integrate it before processing

            questions.append([dtset,qtype,stype,quest_id,question])

            for ith_ref_ans,ref_ans in enumerate(answers_ref):
                ref_answers.append([quest_id,ref_ans])

            for ith_ans,st_answ in enumerate(answers_st):
                st_ans_calif=(grade_st[ith_ans]=="correct")*1
                stud_answers.append([quest_id,st_answ,id_st[ith_ans],st_ans_calif])
    return(questions,ref_answers,stud_answers)

# Trainning Dataset

In [5]:
routes_train=["dataset/training/2way/sciEntsBank/*.xml","dataset/training/2way/beetle/*.xml"]
dataset_train=["TRAIN SciEntsBank","TRAIN Beetle"]

In [6]:
train_questions, train_ref_ans, train_stud_ans = dataset_xml_reader_sep(routes_train,dataset_train)

In [13]:
train_questions_df=pd.DataFrame(train_questions,
                                columns=["dtset","qtype","stype","quest_id","question"])

In [15]:
train_ref_ans_df=pd.DataFrame(train_ref_ans,
                                columns=["quest_id","ref_ans"])

In [17]:
train_stud_ans_df=pd.DataFrame(train_stud_ans,
                                columns=["quest_id","st_answ","id_st","st_ans_calif"])

In [18]:
train_questions_df.to_csv("train_questions.csv", sep=";")
train_ref_ans_df.to_csv("train_ref_ans.csv", sep=";")
train_stud_ans_df.to_csv("train_stud_ans.csv", sep=";")


# Evaluation Dataset

In [19]:
test_routes=["dataset/test/2way/sciEntsBank/test-unseen-answers/*.xml",
             "dataset/test/2way/sciEntsBank/test-unseen-domains/*.xml",
             "dataset/test/2way/sciEntsBank/test-unseen-questions/*.xml",
             "dataset/test/2way/beetle/test-unseen-answers/*.xml",
             "dataset/test/2way/beetle/test-unseen-questions/*.xml"]
test_datasets=["TEST sciEntsBank Unseen Answers",
               "TEST sciEntsBank Unseen Domains",
               "TEST sciEntsBank Unseen Questions",
               "TEST Beetle Unseen Answers",
               "TEST Beetle Unseen Questions"]

In [21]:
test_questions, test_ref_ans, test_stud_ans = dataset_xml_reader_sep(test_routes,test_datasets)

In [22]:
test_questions_df=pd.DataFrame(test_questions,
                                columns=["dtset","qtype","stype","quest_id","question"])

In [23]:
test_ref_ans_df=pd.DataFrame(test_ref_ans,
                                columns=["quest_id","ref_ans"])

In [24]:
test_stud_ans_df=pd.DataFrame(test_stud_ans,
                                columns=["quest_id","st_answ","id_st","st_ans_calif"])

In [25]:
test_questions_df.to_csv("test_questions.csv", sep=";")
test_ref_ans_df.to_csv("test_ref_ans.csv", sep=";")
test_stud_ans_df.to_csv("test_stud_ans.csv", sep=";")
